In [1]:
import SLIM_model
import evaluate
import pandas as pd
import numpy as np
import pickle
import time

from importlib import reload
import optuna

In [2]:
# データロード
user_item_train_df = pd.read_csv('./data/user_item_train.csv')
user_item_test_df = pd.read_csv('./data/user_item_test.csv')
user_list = []
item_list = []
with open('./data/user_list.txt', 'r') as f:
    for l in f:
        user_list.append(l.replace('\n', ''))
        
with open('./data/item_list.txt', 'r') as f:
    for l in f:
        item_list.append(l.replace('\n', ''))

In [5]:
# ハイパラ
# alpha, l1_ratio, lin_model

def time_since(runtime):
    mi = int(runtime / 60)
    sec = int(runtime - mi * 60)
    return (mi, sec)


lin_model = 'elastic'
def objective(trial):
    start = time.time()
    # define model and fit
    alpha = trial.suggest_loguniform('alpha', 1e-6, 1)
    l1_ratio = trial.suggest_uniform('l1_ratio', 0, 1)
    #lin_model = trial.suggest_categorical('lin_model', ['lasso', 'elastic'])
    
    model = SLIM_model.SLIM(alpha, l1_ratio, len(user_list), len(item_list), lin_model=lin_model)
    #model.fit(user_item_train_df)
    model.fit_multi(user_item_train_df)
    #model.load_sim_mat('./sim_mat.txt', user_item_train_df)

    # evaluate
    eval_model = evaluate.Evaluater(user_item_test_df, len(user_list))
    score_sum = 0
    not_count = 0
    for i in range(len(user_list)):
        rec_item_idx = model.pred_ranking(i)
        score = eval_model.topn_precision(rec_item_idx, i)
        if score > 1:
            not_count += 1
            continue
        score_sum += score

    mi, sec = time_since(time.time() - start)
    print('{}m{}sec'.format(mi, sec))
        #if i > 20:
        #    break

    return -1 * (score_sum / (len(user_list) - not_count))

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=20)

17m39sec


[I 2020-07-05 08:42:01,338] Finished trial#0 with value: -0.164546820120712 with parameters: {'alpha': 0.27243969881987223, 'l1_ratio': 0.7327114765483055}. Best is trial#0 with value: -0.164546820120712.


17m36sec


[I 2020-07-05 08:59:37,583] Finished trial#1 with value: -0.164546820120712 with parameters: {'alpha': 0.4381229130460717, 'l1_ratio': 0.8949117371552533}. Best is trial#0 with value: -0.164546820120712.


17m41sec


...

[I 2020-07-05 11:22:35,523] Finished trial#9 with value: -0.13989848068715435 with parameters: {'alpha': 0.00010619086649365309, 'l1_ratio': 0.7535732562493668}. Best is trial#0 with value: -0.164546820120712.


17m46sec


[I 2020-07-05 11:40:22,045] Finished trial#10 with value: -0.1418672629187443 with parameters: {'alpha': 0.002226381260175314, 'l1_ratio': 0.17985204702705093}. Best is trial#0 with value: -0.164546820120712.


17m40sec


[I 2020-07-05 11:58:02,541] Finished trial#11 with value: -0.164546820120712 with parameters: {'alpha': 0.9510804593099471, 'l1_ratio': 0.9970484835423308}. Best is trial#0 with value: -0.164546820120712.


In [ ]:
# パラメータを保存しておく
df = study.trials_dataframe() # pandasのDataFrame形式
df.to_csv('hyparams_result.csv')

In [ ]:
# save best params 
with open('best_param.pickle', 'wb') as f:
    pickle.dump(study.best_params, f)